In [3]:
import random
import numpy as np
import math
from graph_tool import draw, topology, Graph
from deap import base, creator, tools, algorithms
import python_codes.files_operators

# 个体和适应度类
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# 将图转换为个体编码
def graph_to_individual(graph):
    num_vertices = graph.num_vertices()
    individual = [0] * (num_vertices * num_vertices)
    for edge in graph.edges():
        source, target = int(edge.source()), int(edge.target())
        individual[source * num_vertices + target] = 1
        individual[target * num_vertices + source] = 1
    return individual

# 将个体解码为图
def individual_to_graph(individual, num_vertices):
    graph = Graph(directed=False)
    graph.add_vertex(num_vertices)
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            if individual[i * num_vertices + j] == 1:
                graph.add_edge(graph.vertex(i), graph.vertex(j))
    return graph

# 计算欧几里德距离
def euclidean_distance(pos, i, j):
    x_i, y_i = pos[i]
    x_j, y_j = pos[j]
    return math.sqrt((x_i - x_j)**2 + (y_i - y_j)**2)

# 计算所有节点对的总欧几里德距离
def total_euclidean_distance(graph, pos):
    total_distance = 0.0
    num_vertices = graph.num_vertices()
    
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            total_distance += euclidean_distance(pos, i, j)
    
    return total_distance

# 适应度函数
def fitness_function(individual, num_vertices, pos):
    graph = individual_to_graph(individual, num_vertices)
    return total_euclidean_distance(graph, pos),

# 创建初始种群
def create_initial_population(graph, pos, pop_size, num_additional_edges):
    population = []
    for _ in range(pop_size):
        mst, added_edges = create_mst(graph, pos)
        add_random_edges(mst, added_edges, num_additional_edges)
        individual = graph_to_individual(mst)
        population.append(creator.Individual(individual))
    return population

# 生成最小生成树
def create_mst(graph, pos):
    distance_matrix = calculate_all_distances(graph, pos)
    mst = Graph(directed=False)
    mst.add_vertex(graph.num_vertices())
    
    edge_list = [(i, j, distance_matrix[i][j]) for i in range(graph.num_vertices()) for j in range(i + 1, graph.num_vertices())]
    edge_list.sort(key=lambda x: x[2])  # 按距离排序
    
    added_edges = set()
    for edge in edge_list:
        if len(added_edges) >= graph.num_vertices() - 1:
            break
        u, v, _ = edge
        mst.add_edge(u, v)
        added_edges.add((u, v))
        
        if topology.shortest_distance(mst, source=u, target=v) != 1:
            mst.remove_edge(mst.edge(u, v))
            added_edges.remove((u, v))
    
    return mst, added_edges

# 添加随机边
def add_random_edges(graph, existing_edges, num_edges_to_add):
    num_vertices = graph.num_vertices()
    while len(existing_edges) < num_vertices - 1 + num_edges_to_add:
        u, v = random.sample(range(num_vertices), 2)
        if u > v:
            u, v = v, u
        if (u, v) not in existing_edges:
            graph.add_edge(u, v)
            existing_edges.add((u, v))

# 计算所有距离
def calculate_all_distances(graph, pos):
    num_vertices = graph.num_vertices()
    distance_matrix = np.zeros((num_vertices, num_vertices))
    
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            distance_matrix[i][j] = euclidean_distance(pos, i, j)
            distance_matrix[j][i] = distance_matrix[i][j]  # 对称距离
            
    return distance_matrix

# 修复边数的方法
def ensure_edge_count(ind, num_edges):
    current_edges = sum(ind)
    num_vertices = int(math.sqrt(len(ind)))

    if current_edges > num_edges:
        # 边数多于预定的边数，随机移除多余的边
        indices = [i for i, x in enumerate(ind) if x == 1]
        indices_to_remove = random.sample(indices, current_edges - num_edges)
        for index in indices_to_remove:
            ind[index] = 0
    elif current_edges < num_edges:
        # 边数少于预定的边数，随机添加缺少的边
        indices = [i for i, x in enumerate(ind) if x == 0]
        indices_to_add = random.sample(indices, num_edges - current_edges)
        for index in indices_to_add:
            ind[index] = 1

    return ind

# 自定义交叉操作
def custom_crossover(ind1, ind2, num_edges=69):
    num_vertices = int(math.sqrt(len(ind1)))
    
    # 选择两个交叉点
    cx_point1 = random.randint(1, len(ind1) - 2)
    cx_point2 = random.randint(cx_point1, len(ind1) - 1)
    
    # 交换基因片段
    ind1[cx_point1:cx_point2], ind2[cx_point1:cx_point2] = ind2[cx_point1:cx_point2], ind1[cx_point1:cx_point2]
    
    # 确保边数一致
    ind1 = ensure_edge_count(ind1, num_edges)
    ind2 = ensure_edge_count(ind2, num_edges)
    
    return ind1, ind2

# 自定义变异操作
def custom_mutation(individual, indpb=0.05, num_edges=69):
    num_vertices = int(math.sqrt(len(individual)))
    
    # Perform mutation by flipping bits with a certain probability
    for i in range(len(individual)):
        if random.random() < indpb:
            individual[i] = 1 - individual[i]  # Flip the bit
    
    # Ensure the number of edges is correct
    individual = ensure_edge_count(individual, num_edges)
    return individual,

# 读取图对象
filename1 = "Germany50"
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")

# 初始化遗传算法工具箱
toolbox = base.Toolbox()
toolbox.register("individual", graph_to_individual, read_graph)
toolbox.register("population", create_initial_population, read_graph, read_pos, pop_size=100, num_additional_edges=20)
toolbox.register("evaluate", fitness_function, num_vertices=read_graph.num_vertices(), pos=read_pos)
toolbox.register("mate", custom_crossover, num_edges=69)
toolbox.register("mutate", custom_mutation, indpb=0.05, num_edges=69)
toolbox.register("select", tools.selTournament, tournsize=3)

# 设置统计信息和记录本
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min)
stats.register("avg", np.mean)
logbook = tools.Logbook()
logbook.header = ["gen", "min", "avg"]

# 运行遗传算法并记录统计信息
population = toolbox.population()
for gen in range(50):
    population = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, population)
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit
    record = stats.compile(population)
    logbook.record(gen=gen, **record)
    print(logbook.stream)

# 输出结果
best_individual = tools.selBest(population, 1)[0]
optimized_graph = individual_to_graph(best_individual, read_graph.num_vertices())
draw.graph_draw(optimized_graph, read_pos, vertex_text=optimized_graph.vertex_properties["number"], edge_color='red', output_size=(1000, 1000))

# 打印每一代的最小适应度值（即最小总距离长度）
for record in logbook:
    print(f"Generation {record['gen']}: Min Distance = {record['min']}, Avg Distance = {record['avg']}")


gen	min    	avg    
0  	588.291	588.291
1  	588.291	588.291
2  	588.291	588.291
3  	588.291	588.291
4  	588.291	588.291
5  	588.291	588.291
6  	588.291	588.291
7  	588.291	588.291
8  	588.291	588.291
9  	588.291	588.291
10 	588.291	588.291
11 	588.291	588.291
12 	588.291	588.291
13 	588.291	588.291
14 	588.291	588.291
15 	588.291	588.291
16 	588.291	588.291
17 	588.291	588.291
18 	588.291	588.291
19 	588.291	588.291
20 	588.291	588.291
21 	588.291	588.291
22 	588.291	588.291
23 	588.291	588.291
24 	588.291	588.291
25 	588.291	588.291
26 	588.291	588.291
27 	588.291	588.291
28 	588.291	588.291
29 	588.291	588.291
30 	588.291	588.291
31 	588.291	588.291
32 	588.291	588.291
33 	588.291	588.291
34 	588.291	588.291
35 	588.291	588.291
36 	588.291	588.291
37 	588.291	588.291
38 	588.291	588.291
39 	588.291	588.291
40 	588.291	588.291
41 	588.291	588.291
42 	588.291	588.291
43 	588.291	588.291
44 	588.291	588.291
45 	588.291	588.291
46 	588.291	588.291
47 	588.291	588.291
48 	588.291	588.291


KeyError: ('v', 'number')